In [1]:
from langchain.schema import Document
import json

# 파일 경로 
file_paths = {
    "term": "./metadata/term.json",
    "law": "./metadata/load_traffic_law.json",
    "modifier": "./metadata/modiflier.json",
    "car_case": "./metadata/car_to_car.json",
    "law_meta": "./metadata/law.json"
}

# 교통사고 케이스용 필드 상수
CASE_ID = "사건 ID"
CASE_TITLE = "사건 제목"
CASE_SITUATION = "사고상황"
BASE_RATIO = "기본 과실비율"
MODIFIERS = "케이스별 과실비율 조정예시"
LAW_REFERENCES = "관련 법규"
LEGAL_NOTES = "참고 판례"
REASON = "기본 과실비율 해설"

# JSON 로드 함수
def load_json(path):
    with open(path, 'r', encoding='utf-8') as f:
        return json.load(f)

# 리스트형 JSON 변환 (term, modifier, law_meta)
def convert_list_to_documents(data_list, doc_type):
    return [
        Document(page_content=json.dumps(item, ensure_ascii=False), metadata={"type": doc_type})
        for item in data_list
    ]

def convert_car_case_documents(data_list):
    documents = []

    def safe_value(value):
        if isinstance(value, list):
            return ", ".join(map(str, value))
        elif isinstance(value, dict):
            return json.dumps(value, ensure_ascii=False)
        elif value is None:
            return ""  # null도 허용 안 되므로 빈 문자열로 처리
        else:
            return str(value)

    for item in data_list:
        if not isinstance(item, dict):
            continue

        # page_content는 원본 전체 JSON 문자열
        content = json.dumps(item, ensure_ascii=False)

        # 기본 과실비율 해설이 리스트일 수 있음 → 문자열로 병합
        reason = item.get(REASON)
        if isinstance(reason, list):
            reason = "\n".join(map(str, reason))

        metadata = {
            "type": "car_case",
            "id": safe_value(item.get(CASE_ID)),
            "title": safe_value(item.get(CASE_TITLE)),
            "situation": safe_value(item.get(CASE_SITUATION)),
            "base_ratio": safe_value(item.get(BASE_RATIO)),
            "modifiers": safe_value(item.get(MODIFIERS)),
            "law": safe_value(item.get(LAW_REFERENCES)),
            "legal_notes": safe_value(item.get(LEGAL_NOTES)),
            "reason": safe_value(reason)
        }

        documents.append(Document(page_content=content, metadata=metadata))
    return documents

# 도로교통법 law JSON → 문서화
def convert_law_json_to_documents(data_dict):
    documents = []

    def normalize(item):
        return json.dumps(item, ensure_ascii=False) if isinstance(item, dict) else str(item)

    for law_name, content in data_dict.items():
        if isinstance(content, dict):
            for clause, text in content.items():
                lines = [normalize(x) for x in (text if isinstance(text, list) else [text])]
                full_text = f"{law_name} {clause}\n" + "\n".join(lines)
                documents.append(Document(page_content=full_text, metadata={"type": "law"}))
        else:
            lines = [normalize(x) for x in (content if isinstance(content, list) else [content])]
            full_text = f"{law_name}\n" + "\n".join(lines)
            documents.append(Document(page_content=full_text, metadata={"type": "law"}))
    
    return documents


# 문서화 실행
term_docs       = convert_list_to_documents(load_json(file_paths["term"]), "term")
modifier_docs   = convert_list_to_documents(load_json(file_paths["modifier"]), "modifier")
law_meta_docs   = convert_list_to_documents(load_json(file_paths["law_meta"]), "law_metadata")
car_case_docs   = convert_car_case_documents(load_json(file_paths["car_case"]))
law_docs        = convert_law_json_to_documents(load_json(file_paths["law"]))


# 전체 문서 리스트
all_docs = term_docs + modifier_docs + car_case_docs + law_meta_docs + law_docs


In [2]:
from langchain.vectorstores import Chroma  # persist 지원
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 1. 청크 크기 조정 (500~1000 권장)
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
    length_function=len,
    is_separator_regex=False,
)

# 2. 문서 분할
all_splits = text_splitter.split_documents(all_docs)

embedding_model = OpenAIEmbeddings()

# 4. Chroma DB에 배치 처리로 저장
batch_size = 100  # 한 번에 처리할 청크 수
vectorstore = Chroma.from_documents(
    documents=all_splits[:batch_size],  # 첫 배치
    embedding=embedding_model,
    persist_directory="./vectordb"
)

# 남은 청크를 순차적으로 추가
for i in range(batch_size, len(all_splits), batch_size):
    batch = all_splits[i:i+batch_size]
    vectorstore.add_documents(
        documents=batch,
        embedding=embedding_model
    )

vectorstore.persist()  


C:\Users\Playdata\AppData\Local\Temp\ipykernel_12772\3046172222.py:34: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectorstore.persist()


In [3]:
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI
from langchain.vectorstores import Chroma

# 1. LLM 설정
model = ChatOpenAI(model='gpt-4o', temperature=0.5)

# 2. 프롬프트 템플릿
prompt = PromptTemplate(
    template="""
다음 '사고 상황 설명'에 대해 '문서'의 내용만 참고하여 과실 비율 및 법적 판단을 생성해 주세요.

사고 상황: {question}

문서 내용: {context}

답변 형식:
1. 과실비율: A차량 xx% vs B차량 xx%
2. 사고 원인 및 판단 근거:
   - [사고의 전개, 각 차량의 행위, 과실 요소 등을 구체적으로 설명]
3. 관련 법률 조항:
   - [예: 도로교통법 제10조 제2항, 제27조 제1항 등]
4. 참고 판례:
   - [예: 대법원 2023다12345, 서울중앙지법 2022가단123456 등]

조건:
- 반드시 문서 내용만 참고해 판단하세요.
- 법률 조항과 판례가 명시되어 있지 않으면 유사하거나 추정 가능한 근거를 제시해도 됩니다.
- 전체 답변은 포맷에 맞게 간결하고 전문적으로 작성하세요.
""",
    input_variables=["question", "context"]
)

# 3. 리트리버 설정
retriever = vectorstore.as_retriever(search_kwargs={"k": 5})

# 4. QA 체인 재구성 (수정 부분)
qa_chain = RetrievalQA.from_chain_type(
    llm=model,
    retriever=retriever,
    chain_type="stuff",
    chain_type_kwargs={"prompt": prompt},
    return_source_documents=True
)

# 5. 실제 질의 실행
query = "B차량이 비보호좌회전 표지가 없는 교차로에서 녹색등(좌회전 화살표 없음)에 좌회전 시도, 직진하는 A차량과 충돌한 사고가 발생했어. 과실비율이 어떻게 돼??"

res = qa_chain.invoke({"query": query})

# 6. 출력
print("✅ 답변:\n", res["result"])


✅ 답변:
 1. 과실비율: A차량 70% vs B차량 30%

2. 사고 원인 및 판단 근거:
   - 사고는 B차량이 비보호좌회전 표지가 없는 교차로에서 녹색등에 좌회전을 시도하면서 발생하였습니다. A차량은 직진 중이었으며, 교차로에서의 직진 차량에게 우선권이 있습니다. 따라서 B차량은 A차량의 진행을 방해하지 않고 안전하게 좌회전을 할 의무가 있었습니다. 그러나 문서에서 제시된 판례에 따르면, A차량이 전방좌우를 잘 살피지 않고 진입한 경우 B차량의 과실이 30%로 판단된 바 있습니다. 이를 근거로, B차량의 과실을 30%로 판단할 수 있습니다.

3. 관련 법률 조항:
   - 도로교통법 제27조 제1항: 모든 차의 운전자는 교차로에 들어가려 할 때나 교차로 안에서 다른 차와의 진로가 겹칠 때에는 그 차에 진로를 양보하여야 한다.

4. 참고 판례:
   - 대구지방법원 2010가단4991: 신호등 없는 교차로에서의 우회전 및 전방주시의무 관련 판례.


In [4]:
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA

detail_prompt = PromptTemplate(
    template="""
아래 문서 내용을 바탕으로 사용자가 물어본 용어나 법률 조항, 판례에 대해 정확하고 간결하게 설명해 주세요.

질문: {question}

문서: {context}

답변 형식:
- 용어/조항 정의: [정확한 설명]
- 출처가 명시된 경우: 관련 법률/조문 번호/판례명을 반드시 포함

답변:
""",
    input_variables=["question", "context"]
)

detail_chain = RetrievalQA.from_chain_type(
    llm=model,
    retriever=retriever,
    chain_type="stuff",
    chain_type_kwargs={"prompt": detail_prompt}
)

# run은 question 키를 기준으로 실행됨!
res2 = detail_chain.run("보행자우선도로가 뭔지 설명해줘")

print("✅ 기능② 답변:\n", res2)


C:\Users\Playdata\AppData\Local\Temp\ipykernel_12772\1518626063.py:29: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  res2 = detail_chain.run("보행자우선도로가 뭔지 설명해줘")


✅ 기능② 답변:
 - 용어/조항 정의: 보행자우선도로란 보행자가 우선적으로 통행할 수 있도록 지정된 도로를 의미합니다. 이 도로에서는 보행자가 차량보다 우선적으로 통행할 권리를 가지며, 차량 운전자는 보행자의 안전을 최우선으로 고려하여 운전해야 합니다. 보행자우선도로는 주로 보행자 통행이 많은 지역이나 주거 지역에서 설정되며, 보행자의 안전 확보와 생활 환경 개선을 목적으로 합니다.
- 출처가 명시된 경우: 해당 질문과 관련된 법률 조항이나 판례는 제공된 문서에 포함되어 있지 않습니다.


In [5]:
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI

# LLM 설정
model = ChatOpenAI(model='gpt-4o', temperature=0.5)

# 기능 1번. 사고 상황 판단 프롬프트
prompt = PromptTemplate(
    template="""
너는 교통사고 과실 판단 전문가야.
다음 '사고 상황'을 보고, 아래 문서 내용을 기반으로 과실비율을 계산해줘.

사고 상황:
{question}

문서:
{context}

출력 형식:
1. 과실비율: A차량 xx% vs B차량 xx%
2. 판단 근거 요약
3. 적용 법률
4. 참고 판례

조건:
- 반드시 문서 내용 기반으로 판단해
- 문서에 없다면 판단 보류하거나 유사 사례로 근거 설명해
""",
    input_variables=["question", "context"]
)


# 기능 2번. 용어/법률 설명 프롬프트
detail_prompt = PromptTemplate(
    template="""
아래 문서 내용을 바탕으로 사용자가 물어본 용어나 법률 조항, 판례에 대해 정확하고 간결하게 설명해 주세요.

질문: {question}

문서: {context}

답변 형식:
- 용어/조항 정의: [정확한 설명]
- 출처가 명시된 경우: 관련 법률/조문 번호/판례명을 반드시 포함

답변:
""",
    input_variables=["question", "context"]
)

# 리트리버 설정
retriever = vectorstore.as_retriever(search_kwargs={"k": 5})

# 사고 상황 판단 체인
accident_chain = RetrievalQA.from_chain_type(
    llm=model,
    retriever=retriever,
    chain_type="stuff",
    chain_type_kwargs={"prompt": prompt}
)

# 설명용 체인
detail_chain = RetrievalQA.from_chain_type(
    llm=model,
    retriever=retriever,
    chain_type="stuff",
    chain_type_kwargs={"prompt": detail_prompt}
)

def classify_query(user_input: str) -> str:
    accident_keywords = ["충돌", "사고", "과실", "A차량", "B차량", "신호위반", "부딪힘", "진입", "교차로"]
    explanation_keywords = ["무엇", "뭐야", "정의", "설명", "판례", "조항", "법원", "법", "도로교통법" ,"무슨 뜻", "의미", "용어", "이란"]

    # 기능② 설명 우선 처리
    if any(word in user_input for word in explanation_keywords):
        return "detail"
    if any(word in user_input for word in accident_keywords):
        return "accident"
    return "detail"


# 통합 실행 함수
def run_query(user_input: str):
    category = classify_query(user_input)
    if category == "accident":
        result = accident_chain.run(user_input)
        print("✅ 사고 상황 판단 결과:\n", result)
    else:
        result = detail_chain.run(user_input)
        print("📘 용어/법률 설명 결과:\n", result)

### 테스트

In [6]:
run_query("A차량이 적색신호에 직진 중이고, B차량은 좌회전 신호 없이 녹색등에서 좌회전 중 충돌한 사고야. 과실비율은?")

✅ 사고 상황 판단 결과:
 1. 과실비율: A차량 40% vs B차량 60%

2. 판단 근거 요약:
   - A차량은 적색신호에 직진하여 신호위반을 했습니다.
   - B차량은 좌회전 신호 없이 녹색등에서 좌회전하여 신호위반을 했습니다.
   - 문서에서 유사한 상황으로 A차량이 황색신호에 교차로에 진입하고, B차량이 좌회전이 금지된 녹색직진신호에 좌회전한 경우의 기본 과실비율을 40:60으로 설정한 사례를 참고하였습니다.

3. 적용 법률:
   - 도로교통법 제5조(신호 또는 지시에 따를 의무)
   - 도로교통법 제25조(좌회전 및 유턴의 금지)

4. 참고 판례:
   - 문서 내 유사 사례를 기반으로 판단하였으며, 구체적인 판례는 제공되지 않았습니다. 문서에 제시된 기본 과실비율 사례를 참고하였습니다.


In [7]:
run_query("비보호 좌회전이 뭔지 설명해줘")

📘 용어/법률 설명 결과:
 - 용어 정의: 비보호 좌회전은 교차로에서 좌회전 신호 없이 일반 신호에 따라 좌회전을 할 수 있는 상황을 말합니다. 이 경우, 좌회전하는 차량은 반대 방향에서 직진하는 차량의 통행을 방해하지 않도록 주의해야 하며, 안전하게 좌회전을 할 의무가 있습니다.
- 출처: 서울중앙지방법원 2017나65373 판례에 따르면, 비보호 좌회전 중 사고가 발생한 경우 좌회전 차량의 주의 의무 위반이 인정될 수 있으며, 이 판례에서는 A차량의 과실 비율을 80%로 판단하였습니다.


In [11]:
run_query("대법원 92도2077 판결문 내용을 알려줘")

📘 용어/법률 설명 결과:
 - 판례 설명: 대법원 92도2077 판결에서는 차량 운전자가 횡단보도의 신호가 적색인 상태에서 반대 차로에 정지한 차량 뒤로 보행자가 건너올 것을 예상하여 주의의무를 다해야 한다고 할 수 없다고 판시하였습니다. 이는 운전자가 보행자의 행동을 예측할 수 없는 상황에서 법적 책임을 지지 않는다는 취지입니다.


In [12]:
run_query("서울고등법원 2002나57692 판결문 내용을 알려줘")

📘 용어/법률 설명 결과:
 질문하신 "서울고등법원 2002나57692" 판결문에 대한 정보는 제공된 문서에 포함되어 있지 않습니다. 따라서 해당 판결문에 대한 구체적인 내용이나 설명을 제공할 수 없습니다. 

만약 다른 판례나 법률 조항에 대해 궁금한 점이 있으시면 추가로 질문해 주세요. 문서에 포함된 정보를 바탕으로 답변 드리겠습니다.
